the purpose of this code is to match HART stations to the OD Matrix
- then I can identify routes that would use HART stations

this file was originally called 250414_match_hart_v1.ipynb

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

pd.set_option('display.max_columns', None)

In [ ]:
base_dir = os.path.dirname(os.path.dirname(os.getcwd()))
datasets_dir = os.path.join(base_dir, 'datasets')

# Updated file paths
tract_shape_path = os.path.join(datasets_dir, 'tl_2023_15_tract.zip')
tract_gdf = gpd.read_file(tract_shape_path)

station_path = os.path.join(datasets_dir, 'HART_Transit_Stations_PUBLIC_-7935673169750357749.zip')
station_gdf = gpd.read_file(station_path)

In [3]:
# sjoin the stations to tracts
def merge_stations_and_tracts(station_gdf, tract_gdf):
    if station_gdf.crs != tract_gdf.crs:
        station_gdf = station_gdf.to_crs(tract_gdf.crs)
    joined_gdf = gpd.sjoin(station_gdf, tract_gdf, how="left", predicate="within")
    # Clean up column names from the join operation
    if 'index_right' in joined_gdf.columns:
        joined_gdf = joined_gdf.drop(columns=['index_right'])
    return joined_gdf

In [4]:
stations_with_tracts = merge_stations_and_tracts(station_gdf, tract_gdf)
print(f"Number of stations: {len(stations_with_tracts)}")
print(f"Number of tracts covered: {len(stations_with_tracts['GEOID'].unique())}")
station_tract_list = stations_with_tracts['GEOID'].unique().tolist()

Number of stations: 21
Number of tracts covered: 18


read od matrix and filter for rows that end in the station tracts

In [ ]:
base_dir = os.path.dirname(os.path.dirname(os.getcwd()))
processed_dir = os.path.join(base_dir, 'datasets', 'processed')
od_path = os.path.join(processed_dir, 'od_matrix.csv')

od_df = pd.read_csv(od_path)
od_df['origin_tract'] = od_df['h_geocode'].astype('str').str[:11]
od_df['destination_tract'] = od_df['w_geocode'].astype('str').str[:11]

od_df_filtered = od_df[od_df['destination_tract'].isin(station_tract_list)].copy()

there are ~52k vehicles that end in the station tracts

find every road link for each vehicle

In [ ]:
import networkx as nx
import json
from collections import Counter

In [8]:
def read_edges_path():
    edges_path = os.path.join(os.path.dirname(os.getcwd()), 'datasets', 'processed', 'edges.gpkg')
    return edges_path

def read_edges(edges_path):
    edges = gpd.read_file(edges_path)
    return edges

def create_network_graph(edges_df):
    """Create directed graph from edges dataframe using free_flow_travel_time as weight"""
    G = nx.DiGraph()
    
    # Add edges with their attributes
    for _, row in edges_df.iterrows():
        G.add_edge(
            str(row['Node1']),
            str(row['Node2']),
            weight=row['free_flow_travel_time'],  # Using free flow travel time as weight
            length=row['length'],
            length_miles=row['length_miles'] if 'length_miles' in row else None,
            FFS=row['FFS'] if 'FFS' in row else None, 
            link_id=row['link_id']
        )
    
    print(f"Network created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

def convert_node_paths_to_links(vehicles_df, edges_df):
    """
    Convert node paths to link sequences using the edges dataframe.
    Returns the vehicles_df with an additional 'route_links' column.
    """
    # Create node pair to link_id lookup
    link_lookup = {(str(row['Node1']), str(row['Node2'])): row['link_id'] 
                  for _, row in edges_df.iterrows()}
    
    # Initialize route_links column
    vehicles_df['route_links'] = None
    vehicles_df['conversion_status'] = 'pending'
    
    # Process only vehicles with successful node paths
    mask = vehicles_df['routing_status'] == 'success'
    print(f"Converting node paths to links for {mask.sum()} vehicles...")
    
    success_count = 0
    fail_count = 0
    
    for idx, row in vehicles_df[mask].iterrows():
        try:
            node_path = json.loads(row['node_path'])
            path_links = []
            path_is_continuous = True
            
            for i in range(len(node_path) - 1):
                node_pair = (str(node_path[i]), str(node_path[i + 1]))
                if node_pair not in link_lookup:
                    path_is_continuous = False
                    break
                path_links.append(link_lookup[node_pair])
            
            if path_is_continuous and path_links:
                vehicles_df.loc[idx, 'route_links'] = json.dumps(path_links)
                vehicles_df.loc[idx, 'conversion_status'] = 'success'
                success_count += 1
            else:
                vehicles_df.loc[idx, 'conversion_status'] = 'discontinuous'
                fail_count += 1
                
        except Exception as e:
            vehicles_df.loc[idx, 'conversion_status'] = f'error: {str(e)}'
            fail_count += 1
            
        # Update progress
        if (success_count + fail_count) % 1000 == 0:
            print(f"Processed {success_count + fail_count} vehicles...")
    
    print(f"\nLink Conversion Summary:")
    print(f"Successful conversions: {success_count}")
    print(f"Failed conversions: {fail_count}")
    
    return vehicles_df

In [9]:
def calculate_node_paths(G, od_matrix):
    """
    Calculate the node-to-node paths for each origin-destination pair.
    Returns the od_matrix with an additional 'node_path' column.
    """
    # Initialize output dataframe
    vehicles_df = od_matrix.copy()
    vehicles_df['node_path'] = None
    vehicles_df['routing_status'] = 'pending'
    
    # Calculate paths for unique OD pairs
    unique_od_pairs = od_matrix.groupby(['origin_node', 'destination_node']).size().reset_index()
    print(f"Calculating paths for {len(unique_od_pairs)} unique OD pairs...")
    
    # Create a progress counter
    processed = 0
    for _, row in unique_od_pairs.iterrows():
        origin = str(row['origin_node'])
        destination = str(row['destination_node'])
        
        try:
            # Find shortest path
            path_nodes = nx.shortest_path(G, origin, destination, weight='weight')
            
            # Update all vehicles with this OD pair
            mask = ((vehicles_df['origin_node'] == row['origin_node']) & 
                   (vehicles_df['destination_node'] == row['destination_node']))
            vehicles_df.loc[mask, 'node_path'] = json.dumps(path_nodes)  # Store as JSON string temporarily
            vehicles_df.loc[mask, 'routing_status'] = 'success'
            
        except nx.NetworkXNoPath:
            mask = ((vehicles_df['origin_node'] == row['origin_node']) & 
                   (vehicles_df['destination_node'] == row['destination_node']))
            vehicles_df.loc[mask, 'routing_status'] = 'no_path'
        
        # Update progress
        processed += 1
        if processed % 100 == 0:
            print(f"Processed {processed} of {len(unique_od_pairs)} OD pairs...")
    
    # Print summary
    success_count = (vehicles_df['routing_status'] == 'success').sum()
    fail_count = (vehicles_df['routing_status'] != 'success').sum()
    print(f"\nPath Calculation Summary:")
    print(f"Successful paths: {success_count}")
    print(f"Failed paths: {fail_count}")
    
    return vehicles_df

def create_route_assignment_data(od_matrix):
    # Read input data
    print("Reading input data...")
    
    edges_path = read_edges_path()
    edges_df = read_edges(edges_path)
    
    # Make sure edges have the right datatypes
    edges_df['link_id'] = edges_df['link_id'].astype(str)
    edges_df['Node1'] = edges_df['Node1'].astype(str)
    edges_df['Node2'] = edges_df['Node2'].astype(str)
    
    # Ensure free_flow_travel_time exists in seconds
    if 'free_flow_travel_time' not in edges_df.columns:
        if 'FFS' in edges_df.columns and 'length_miles' in edges_df.columns:
            # Calculate free flow travel time in seconds
            # FFS is in mph, length_miles is in miles
            # Time = distance / speed, converted to seconds
            edges_df['free_flow_travel_time'] = (edges_df['length_miles'] / edges_df['FFS']) * 3600
        else:
            raise ValueError("Cannot calculate free_flow_travel_time - missing FFS or length_miles columns")
    
    # Create network graph
    G = create_network_graph(edges_df)
    
    # Calculate optimal routes
    vehicles_with_paths = calculate_node_paths(G, od_matrix)
    
    # Convert node paths to link sequences
    vehicles_with_links = convert_node_paths_to_links(vehicles_with_paths, edges_df)

    # Convert the DataFrame to a list of dictionaries, preserving the actual lists
    # instead of JSON strings in the node_path and route_links columns
    result = []
    for _, row in vehicles_with_links.iterrows():
        vehicle_dict = row.to_dict()
        
        # Parse JSON string back to list if not None
        if vehicle_dict.get('node_path'):
            vehicle_dict['node_path'] = json.loads(vehicle_dict['node_path'])
        
        if vehicle_dict.get('route_links'):
            vehicle_dict['route_links'] = json.loads(vehicle_dict['route_links'])
        
        result.append(vehicle_dict)
    
    return result

find the earliest node in the list that falls within one of the highway tracts

In [ ]:
nodes_path = os.path.join(processed_dir, 'nodes.gpkg')
nodes_gdf = gpd.read_file(nodes_path)

# find tract for these nodes
nodes_tracts = merge_stations_and_tracts(nodes_gdf, tract_gdf)[['Node', 'GEOID']]
# filter for station tracts
station_nodes_tract_level = nodes_tracts[nodes_tracts['GEOID'].isin(station_tract_list)]
station_nodes_list = station_nodes_tract_level['Node'].unique().tolist()

In [11]:
def add_station_node_indicators(vehicles_json, station_nodes_list):
    # Convert station_nodes_list to string for consistent comparison
    station_nodes_set = set(str(node) for node in station_nodes_list)
    
    for vehicle in vehicles_json:
        # Skip vehicles with no path
        if not vehicle.get('node_path'):
            vehicle['station_node_indicators'] = []
            continue
            
        # Create binary indicators
        node_path = vehicle['node_path']
        station_indicators = [1 if str(node) in station_nodes_set else 0 for node in node_path]
        
        # Add to vehicle dictionary
        vehicle['station_node_indicators'] = station_indicators
    
    return vehicles_json

def add_node_statistics(vehicles_json):
    """
    Add statistics about nodes in the path:
    1. total_nodes: Total number of nodes in the path
    2. earliest_station_position: Index of first station node (or -1 if none found)
    3. earliest_station_node: The first station node in the path (or None if none found)
    
    Parameters:
    vehicles_json (list): List of vehicle dictionaries with node_path and station_node_indicators
    
    Returns:
    list: Updated vehicles_json with additional statistics
    """
    for vehicle in vehicles_json:
        # Skip vehicles with no path
        if not vehicle.get('node_path'):
            vehicle['total_nodes'] = 0
            vehicle['earliest_station_position'] = -1
            vehicle['earliest_station_node'] = None
            continue
            
        # Add total count of nodes
        vehicle['total_nodes'] = len(vehicle['node_path'])
        
        # Find position of earliest station node (first 1 in the indicator list)
        indicators = vehicle.get('station_node_indicators', [])
        node_path = vehicle.get('node_path', [])
        
        try:
            # Find the index of the first 1 (if any)
            earliest_station = indicators.index(1)
            vehicle['earliest_station_position'] = earliest_station
            
            # Get the actual node value at this position
            vehicle['earliest_station_node'] = node_path[earliest_station]
        except ValueError:
            # If no 1 found in the list, set to -1 and None
            vehicle['earliest_station_position'] = -1
            vehicle['earliest_station_node'] = None
    
    return vehicles_json

In [1]:
# route_assignment_json = create_route_assignment_data(od_df_filtered)
# route_assignment_json = add_station_node_indicators(route_assignment_json, station_nodes_list)
# route_assignment_json = add_node_statistics(route_assignment_json)

# save_path = os.path.join(base_dir, 'datasets')
# output_path = os.path.join(save_path, "250414_hart_mapping.json")

# def save_route_data_to_json(vehicles_json, output_path):
#     # Ensure the directory exists
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
#     # Save the JSON data with nice formatting
#     with open(output_path, 'w') as json_file:
#         json.dump(vehicles_json, json_file, indent=2)
    
#     print(f"Data successfully saved to: {output_path}")
#     return output_path

# save_route_data_to_json(route_assignment_json, output_path)

### read back the json and merge it into the od matrix

In [ ]:
read_path = os.path.join(processed_dir, '250414_hart_mapping.json')
with open(read_path, 'r') as json_file:
    route_assignment_json = json.load(json_file)
# Convert to DataFrame for easier manipulation
route_assignment_df = pd.json_normalize(route_assignment_json)[['person_id', 'total_nodes', 'earliest_station_position', 'earliest_station_node']].copy()

# merge with od matrix
route_assignment_df['person_id'] = route_assignment_df['person_id'].astype(str)
od_df['person_id'] = od_df['person_id'].astype(str)
merged_od_df = pd.merge(od_df, route_assignment_df, left_on='person_id', right_on='person_id', how='left')

save_path = processed_dir
# merged_od_df.to_csv(os.path.join(save_path, "250414_od_matrix_with_hart_designation.csv"), index=False)